# Exercises C: Solutions
_Version: October 31, 2020, see_ [PyEcon.org](https://pyecon.org).

### Exercise 1: Determinant of a matrix
Create a function that recursively calculates the determinant of a matrix $M \in \mathcal{R}^{n\times n}$. The recursive definition is given by

$$\det(M) = \sum_{j = 1}^n (-1)^{1 + j} \cdot M_{1, j} \cdot \det(M_{-1, -j}),$$

where $M_{-1, -j}$ denotes the matrix $M$ without the first row and the $j$th column.

1. Create the determinant function.
2. Calculate the determinant of
$
M = \begin{bmatrix}
1 & 4 & 8 \\
2 & 2 & 9\\
3 & 7 & 1
\end{bmatrix}
$.
3. Compare your determinant function with `numpy.linalg.det()`.

In [ ]:
import numpy as np
import numpy.linalg as nplin


def own_det(x):
    """Computes the determinant of M"""
    n, m = x.shape
    if n != m:
        raise ValueError("M is not a square matrix")
    if n == 1:
        return x
    else:
        det = 0
        for j in range(n):
            x_new = x.copy()
            x_new = np.delete(x_new, 0, 0)
            x_new = np.delete(x_new, j, 1)
            det = det + x[0, j] * own_det(x_new) * (-1)**(1 + (j+1))
        return det


m = np.array([[1, 4, 8],
              [2, 2, 9],
              [3, 7, 1]])
own_determinant = own_det(m)
own_determinant

In [ ]:
# compare determinants
determinant = nplin.det(m)
np.allclose(own_determinant, determinant)

In [ ]:
r = np.random.rand(4, 4)
np.allclose(own_det(r), nplin.det(r))

### Exercise 2: Inverse of a matrix
Build a function that computes the inverse of matrix $M$ using the _adjoint method_, and your determinant function from Exercise 1. An element $C_{ij}$ of $C$, the cofactor matrix of $M$, can be written as

$$C_{ij} = (-1)^{i + j} \cdot \det(M_{-i,-j}),$$

and the inverse can be calculated by

$$M^{-1} = \frac{1}{\det(M)} \cdot C',$$

where $C'$, the transposed matrix $C$, is the so-called adjugate matrix of $A$.

1. Create the inverse function.
2. Calculate the inverse of
$
M = \begin{bmatrix}
1 & 4 & 8 \\
2 & 2 & 9\\
3 & 7 & 1
\end{bmatrix}
$.
3. Compare your inverse function with `numpy.linalg.inv()`.

In [ ]:
import numpy as np
import numpy.linalg as nplin


def own_inv(x):
    """Computes the inverse of M"""
    n, m = x.shape
    if n != m:
        raise ValueError("M is not a square matrix")
    c = np.empty([n, n])
    for i in range(n):
        for j in range(n):
            minor = x.copy()
            minor = np.delete(minor, i, 0)
            minor = np.delete(minor, j, 1)
            c[i, j] = (-1)**((i+1) + (j+1)) * own_det(minor)
    return (1 / own_det(x)) * c.T


m = np.array([[1, 4, 8],
              [2, 2, 9],
              [3, 7, 1]])
my_inverse = own_inv(m)
my_inverse

In [ ]:
inverse = nplin.inv(m)
np.allclose(my_inverse, inverse)

In [ ]:
r = np.random.rand(4, 4)
np.allclose(own_inv(r), nplin.inv(r))

### Exercise 3: Applying and observing the eigendecomposition

Consider the following matrix:

$$
X = \begin{bmatrix}
15.00 & 24.25 & 12.25 \\
24.25 & 44.75 & 32.50 \\
12.25 & 32.50 & 41.25
\end{bmatrix}
$$

The eigendecomposition of a symmetric matrix is $X = E\Lambda E'$, where $E$ is the matrix of the corresponding eigenvectors and $\Lambda$ is the diagonal matrix of eigenvalues. In the case of a symmetric matrix, $E$ is orthogonal, i.e., $EE' = I$, where $I$ is the identity matrix of appropriate dimensions.

1. Determine the diagonal matrix of eigenvalues and the matrix of eigenvectors for $X$ with `numpy.linalg`.
1. From your results of the eigendecomposition use the formula above and recompute $X$. Compare your results.
1. Calculate $X^2$. The calculate $E \Lambda^2 E'$. Compare your results. What do you observe?
1. Compute the inverse by $X = E \Lambda^{-1} E'$ and compare it to the inverse of $X$, e.g., by employing `numpy.linalg.inv()`.
1. Compute $E_{(1)}' E_{(1)}$ and $E_{(1)} E_{(1)}'$, where $E_{(1)}$ represents the first column of matrix $E$.
1. Calculate $\sum_{i = 1}^n \Lambda_{ii} E_{(i)} E_{(i)}'$, where $n$ refers to the number of columns of $X$ and $\Lambda_{ii}$ is the $i$th diagonal element of $\Lambda$.

In [ ]:
import numpy as np
import numpy.linalg as nplin

# Eigendecomposition of matrix X
x = np.array([[15, 24.25, 12.25],
              [24.25, 44.75, 32.5],
              [12.25, 32.5, 41.25]])
lv, e = nplin.eig(x)
lm = np.diag(lv)
print('L = \n', lm)
print('E = \n', e)

In [ ]:
# Verifying the eigendecomposition
x_eig = e @ lm @ e.T
# Alternative
# x_eig = np.dot(np.dot(e, l), e.T)
np.allclose(x, x_eig)

In [ ]:
# Squaring a symmetric matrix by squaring its eigenvalue matrix
x2 = x @ x
x_eig_sq = e @ (lm @ lm) @ e.T
np.allclose(x2, x_eig_sq)

In [ ]:
# Inverting a symmetric matrix by inverting its eigenvalue matrix
x_inv = e @ nplin.inv(lm) @ e.T
np.allclose(x_inv, nplin.inv(x))

In [ ]:
# Two distinct vectors of the eigenvector matrix are orthogonal ...
np.allclose(e[:, 0] @ e[:, 1], 0)

In [ ]:
# ...and the scalar product of a vector with itself is (in theory) 1
e[:, 0] @ e[:, 0]

In [ ]:
# Without re-dimensioning the vectors, np.outer() computes the 'outer product'
np.outer(e[:, 0], e[:, 0])

In [ ]:
# Another way for reconstructing the original matrix by its eigendecomposition
m = np.zeros(e.shape)
for i in range(e.shape[1]):
    m = m + lm[i, i] * np.outer(e[:, i], e[:, i])
np.allclose(m, x)

### Exercise 4: Password generation and encryption
1. Write a function that creates a password by randomly picking characters, i.e., letters, numbers, etc.
1. Modify your function and include an option so that the function generates passwords where each character is unique.
1. Encrypt and decrypt this text: "MONTY PYTHON"
    1. Use the function of 2. to generate a 27 digits password.
    1. Write an encoding function that you can use to encrypt uppercase letters texts and spaces.
    1. Write a decryption function for decoding the previously encrypted text.

 _Hints:_
 * _You could use the modules random or numpy.random._
 * _You might benefit from studying the Python library on string methods._

In [ ]:
import random


def gen_password(digits=12):
    signs = "abcdefghijklmnopqrstuvwxyzäöü"
    signs += signs.upper()
    signs += "! '§$%&/()=?#+*~_<>|^°"
    signs += "1234567890"
    return "".join(random.choices(signs, k=digits))


gen_password(20)

In [ ]:
def gen_password(digits=12, unique=False):
    signs = "abcdefghijklmnopqrstuvwxyzäöü"
    signs += signs.upper()
    signs += "! '§$%&/()=?#+*~_<>|^°"
    signs += "1234567890"
    if unique:
        return "".join(random.sample(signs, k=digits))
    else:
        return "".join(random.choices(signs, k=digits))


key = gen_password(27, unique=True)
key

In [ ]:
def encrypt_text(text, key, decrypt=False):
    alphabet = "abcdefghijklmnopqrstuvwxyz ".upper()
    if len(key) != len(alphabet):
        raise ValueError(f"'key' length must be {len(alphabet)}")
    if decrypt:
        alphabet, key = (key, alphabet)
    return text.translate(str.maketrans(alphabet, key))


text = "MONTY PYTHON"
encrypted = encrypt_text(text, key)
encrypted

In [ ]:
decrypted = encrypt_text(encrypted, key, decrypt=True)
decrypted

### Exercise 5: Simulation and estimation techniques
Simulate data from a random variable $Y$, which can be thought to be the level of production of some country, with the following properties:

$Y \sim \mathcal{\ln N}(\mu,\,\sigma^{2})$,

with

$E(Y) = AK^{1-\alpha}L^\alpha$,

where

$A = 1$ (Technology),

$K  \sim \mathcal{U}(30, 50)$ (Capital),

$L  \sim \mathcal{U}(20, 40)$ (Labour).

Note that $\mu = \ln E(Y) - \sigma^{2} / 2$. See the Log-normal distribution for details.

#### I. Simulate data

1. Set a seed so that your results are reproducible and define the parameters $\sigma^{2} = 0.5$ and $\alpha = 0.7$.
1. Draw $n = 100000$ observations for $K$, $L$ and $Y$.
1. Build a data frame of the observed vectors $K$, $L$ and $Y$.
1. Add the logarithms of all columns to the dataset and name them $y$, $k$ and $l$.


#### II. Ordinary least squares (OLS) estimation

Validate your simulation by conducting an OLS estimation with the logarithmized data:

Estimate the linear regression model

$$
\boldsymbol{y} = X\boldsymbol{\beta} + \boldsymbol{e},
$$

where $X$ contains $\boldsymbol{k}$ and $\boldsymbol{l}$ as its columns as well as an intercept (a column of ones). $p$ is the number of columns of $X$.

The OLS estimators are defined as

$$
\boldsymbol{b} = (X'X)^{-1}X'\boldsymbol{y},
$$

and

$$
\hat{\sigma}^2 = \frac{\hat{\boldsymbol{e}}'\hat{\boldsymbol{e}}}{n - p},
$$

where $\hat{\boldsymbol{e}} = \boldsymbol{y}-X\boldsymbol{b}$.


#### III Non-linear estimation algorithm

Implement the Newton-Raphson algorithm based on $ln Y = \boldsymbol{y}$, i.e., using the normally distributed data. The algorithm proceeds as follows:

$$
\boldsymbol{\theta}_{j+1} = \boldsymbol{\theta}_j - H^{-1}\boldsymbol{s},
$$

where $\boldsymbol{\theta} = [\boldsymbol{\beta}', \sigma^2]'$ and $H = \frac{\delta^2 \ln L}{\delta \boldsymbol{\theta} \delta \boldsymbol{\theta}'}$ and $\boldsymbol{s} = \frac{\delta \ln L}{\delta \boldsymbol{\theta}}$ are the _Hessian matrix_ and the gradient (here _score_), i.e., the matrix of the second derivatives and the vector of the first derivatives of the target function.

In this case, the target is the logarithmized likelihood function

$$
\ln L \left(\boldsymbol{\theta}| \boldsymbol{y}, X\right) =  -\frac{n}{2}\ln \left(2\pi\right) -\frac{n}{2}\ln \left(\sigma^2 \right) - \frac{\left(\boldsymbol{y}-X\boldsymbol{\beta} \right)'\left(\boldsymbol{y}-X\boldsymbol{\beta} \right)}{2\sigma^2},
$$

with

$$
H =
\begin{bmatrix}
-\frac{1}{\sigma^2}X'X & -\frac{1}{\sigma^4}X'\boldsymbol{e} \\
-\frac{1}{\sigma^4}\boldsymbol{e}'X & \frac{n}{2\sigma^4} - \frac{1}{\sigma^6}\boldsymbol{e}'\boldsymbol{e}
\end{bmatrix},
$$

and

$$
\boldsymbol{s} =
\begin{bmatrix}
\frac{1}{\sigma^2}X'\boldsymbol{y} - \frac{1}{\sigma^2}X'X\boldsymbol{\beta} \\
- \frac{n}{2\sigma^2} + \frac{1}{2\sigma^4}\boldsymbol{e}'\boldsymbol{e}
\end{bmatrix},
$$

where $\boldsymbol{e} = \boldsymbol{y}-X\boldsymbol{\beta}$.

* Start with some initial guess $\boldsymbol{\theta}_0 = (-0.3, 0.2, 0.8, 0.6)$.
* Make sure the variance, i.e., the estimator for $\theta_4 = \sigma^2$, cannot turn negative in any iteration step.
* Stop the algorithm if $\text{abs}(\boldsymbol{\theta}_{j+1} - \boldsymbol{\theta}_j) < 0.00001 \cdot  \boldsymbol{1}$.
* Compare your estimates obtained from the Newton-Raphson algorithm to the results of the analytical solution $\boldsymbol{b}$.

In [ ]:
import numpy as np
import numpy.linalg as nplin
import pandas as pd

# Setting random seed
np.random.seed(1337)

# Setting parameters
n = 100000
sigma2 = 0.5
alpha = 0.7

# Simulating data
a = 1
kv = 30 + np.random.rand(n) * (50 - 30)
lv = 20 + np.random.rand(n) * (40 - 20)
mu = np.log(a * kv**(1-alpha) * lv**alpha) - sigma2/2
y = np.random.lognormal(mu, np.sqrt(sigma2), n)
y

In [ ]:
# Building the DataFrame
data = pd.DataFrame({"Y": y, "K": kv, "L": lv})
data.head()

In [ ]:
# Extending the DataFrame
data["k"] = np.log(data["K"])
data["l"] = np.log(data["L"])
data["y"] = np.log(data["Y"])
data.head()

In [ ]:
# OLS estimation
y = data["y"].values
x = np.array([np.ones(n), data["k"], data["l"]]).T
n, p = x.shape
b = nplin.inv(x.T @ x) @ (x.T @ y)
e = y - x @ b
sigma2_est = (e @ e) / (n - p)
print("b = \n", b)
print("ln(A) = \n", np.log(a))
print("\nNote that b[0] is not an estimator for ln{A}"
      " but for ln{A} - sigma2/2.")
print("ln(A) - sigma^2/2 = \n", np.log(a) - sigma2/2, "\n")
print("sigma2_est = \n", sigma2_est)

In [ ]:
# Newton-Raphson non-linear optimization


def loglike(theta, y, x):
    n = y.shape[0]
    e = y - x @ theta[:-1]
    llv = -(n/2)*np.log(2 * np.pi) - (n/2)*np.log(theta[-1]) \
          - (e.T @ e) / (2 * theta[-1])
    return llv


def score(theta, y, x):
    n = y.shape[0]
    e = y - x @ theta[:-1]
    s1 = (x.T @ y - x.T @ x @ theta[:-1]) / theta[-1]
    s2 = np.atleast_1d(-n / (2 * theta[-1]) + (e.T @ e) / (2 * theta[-1]**2))
    return np.concatenate([s1, s2])


def hessian(theta, y, x):
    n = y.shape[0]
    e = y - x @ theta[:-1]
    h11 = -(x.T @ x) / theta[-1]
    h12 = np.atleast_2d(-(x.T @ e) / theta[-1]**2).T
    h22 = n / (2 * theta[-1]**2) - (e.T @ e) / theta[-1]**3
    return np.block([[h11, h12],
                     [h12.T, h22]])


def maximize_loglike(theta0, y, x, eps=1e-5):
    theta = theta0
    while True:
        pre = theta
        theta = theta - nplin.inv(hessian(theta, y, x)) @ score(theta, y, x)
        if theta[-1] <= 0:
            theta[-1] = 1e-8
        if np.all(np.abs(theta - pre) < eps):
            break
        # optionally printing process
        print(theta)
    return theta, loglike(theta, y, x)


# maximize multivariate normal log-likelihood
theta0 = np.array([-0.3, 0.2, 0.8, 0.6])
theta_max, llv_max = maximize_loglike(theta0, y, x)
print("loglike_ml: \n", llv_max)
print("b_ml = \n", theta_max[:-1])
print("sigma2_ml = \n", theta_max[-1])

In [ ]:
# compare with OLS estimate
np.allclose(np.block([b, sigma2_est]), theta_max, atol=1e-4, rtol=0)